# EuroPython 2015 booklet program listing

In [26]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 99999;
//increase max size of output area

<IPython.core.display.Javascript object>

In [1]:
import json
import datetime as dt
from   operator import itemgetter

from IPython.display import display, HTML
from IPython.nbconvert.filters.markdown import markdown2html

In [2]:
def to_str(bytes_or_str):
    if isinstance(bytes_or_str, bytes):
        value = bytes_or_str.decode('utf-8')
    else:
        value = bytes_or_str
    return value  # Instance of str”


def to_bytes(bytes_or_str):
    if isinstance(bytes_or_str, str):
        value = bytes_or_str.encode('utf-8')
    else:
        value = bytes_or_str
    return value # Instance of bytes”


# Python 2
def to_unicode(unicode_or_str):
    if isinstance(unicode_or_str, str):
        value = unicode_or_str.decode('utf-8')
    else:
        value = unicode_or_str
    return value # Instance of unicode

# Python 2
def to_str(unicode_or_str):
    if isinstance(unicode_or_str, unicode):
        value = unicode_or_str.encode('utf-8')
    else:
        value = unicode_or_str
    return value # Instance of str”

In [3]:
talk_sessions = json.load(open('../static/data/talks.json'))
talks_admin_url = 'https://ep2015.europython.eu/admin/conference/talk'

In [65]:
show = lambda s: display(HTML(s))

def ordinal(n):
    if 10 <= n % 100 < 20:
        return str(n) + 'th'
    else:
        return  str(n) + {1 : 'st', 2 : 'nd', 3 : 'rd'}.get(n % 10, "th")

def talk_schedule(start, end):

    input_format  = "%Y-%m-%d %H:%M:%S"
    output_format_day = "%A, %B"
    output_format_time = "%H:%M"
    
    output_date = lambda d: "{} {} at {}".format(d.strftime(output_format_day), 
                                                 ordinal(int(d.strftime('%d'))),
                                                 d.strftime(output_format_time))
    
    start_date = dt.datetime.strptime(start, input_format)
    end_date   = dt.datetime.strptime(end  , input_format)

    return output_date(start_date), output_date(end_date)


def show_talk(talk, show_duration=True, show_link_to_admin=True):
    speakers  = to_str(talk['speakers'])
    title     = to_str(talk['title'])
    #abstract  = talk['abstracts'][0]
    room      = to_str(talk.get('track_title', '').split(', ')[0])
    timerange = to_str(talk.get('timerange', '').split(';')[0])
    
    show('<h2>{}</h2>'.format(title))
    
    if show_link_to_admin:
        talk_admin_url = talks_admin_url + '/{}'.format(talk['id'])
        show('<a href={0}>{0}</a>'.format(talk_admin_url))
    
    if show_duration:
        duration = '{} mins.'.format(talk['duration'])
    else:
        duration = ''

    timerange = talk['timerange'].split(';')[0]
    try:
        start, end = talk_schedule(*timerange.split(', '))
    except:
        start, end = ('', '')

    if start:
        schedule  = '<p>'
        schedule += '{} in {}'.format(start, room)
        if show_duration:
            schedule += ' ({})'.format(duration)
        schedule += '</p>'

        show(schedule)
    
    show('<h3><i>{}</i></h2>'.format(speakers))
    #show('<p>{}</p>'.format(markdown2html(abstract)))
    show('<br/>')

    
    
def talk_schedule_2(start, end):

    input_format  = "%Y-%m-%d %H:%M:%S"
    output_format_day = "%A, %B"
    output_format_time = "%H:%M"
    
    output_date = lambda d: "{} {} at {}".format(d.strftime(output_format_day), 
                                                 ordinal(int(d.strftime('%d'))),
                                                 d.strftime(output_format_time))
    
    output_day = lambda d: "{}".format(d.strftime("%d"))
    
    output_hour = lambda d: "{}".format(d.strftime(output_format_time))
    
    start_date = dt.datetime.strptime(start, input_format)
    end_date   = dt.datetime.strptime(end  , input_format)

    return output_day(start_date), output_hour(start_date), output_day(end_date), output_hour(end_date)


def get_all_talks_from_room(room):
    ts = []
    for session_name in session_names:
        talks = talk_sessions[session_name]
        talks = [talks[talk_id] for talk_id in talks]
        talks = sorted(talks, key=itemgetter('title'))
        for talk in talks:
            if (to_str(talk.get('track_title', '').split(', ')[0]) == room):
                speakers  = to_str(talk['speakers'])
                title     = to_str(talk['title'])
                timerange = to_str(talk.get('timerange', '').split(';')[0])
                try:
                    start_date, start_hour, end_day, end_hour = talk_schedule_2(*timerange.split(', '))
                except:
                    start, end = ('', '')
       
                t = { 'speaker': speakers,
                    'title': title,
                    'start_date': start_date,
                    'start_time': start_hour,
                     'end_date': end_day,
                    'end_time': end_hour}
                ts.append(t)
    return(ts)

In [5]:
session_names = ['Keynotes', 'Talks', 'Trainings', 'Help desks',
                 'EuroPython sessions', 'Other sessions', 'Poster sessions']

In [6]:
for session_name in session_names:
    show('<h1>{}</h1>'.format(session_name))
    talks = talk_sessions[session_name]
    talks = [talks[talk_id] for talk_id in talks]
    talks = sorted(talks, key=itemgetter('title'))
    for talk in talks:
        show_talk(talk, show_duration=False, show_link_to_admin=False)

In [117]:
def next_session(room, time, quant=1):
    input_format = "%H:%M"    
    ts = get_all_talks_from_room(room)
    ts = sorted(ts, key=itemgetter("start_date", "start_time"))
    next_session = []
    for talk in ts:
        if int(talk["start_date"]) == time.day:
            start_dt_hour = dt.datetime.strptime(talk["start_time"], input_format)
            if start_dt_hour.time() > time.time():
                next_session.append(talk)
    return(next_session[0:quant])




In [66]:
ts = get_all_talks_from_room("Google Room")

In [118]:
a = sorted(ts, key=itemgetter('start_date'))

n = next_session("Google Room", dt.datetime(2015,7,21,9,30),3)

print(n)

[{'end_date': '21', 'title': 'Advanced Web Scraping', 'start_time': '11:00', 'speaker': 'Shane Evans', 'end_time': '11:45', 'start_date': '21'}, {'end_date': '21', 'title': 'Dive into Scrapy', 'start_time': '11:45', 'speaker': 'Juan Riaza', 'end_time': '12:30', 'start_date': '21'}, {'end_date': '21', 'title': 'Nameko for Microservices', 'start_time': '12:30', 'speaker': 'Matt Bennett', 'end_time': '13:15', 'start_date': '21'}]


In [12]:
input_format  = "%A, %B %d %H:%M"
[t for t in ts if dt.datetime.strptime(t['start']) == 21 ]

TypeError: strptime() takes exactly 2 arguments (1 given)

In [59]:
ts[0]['start']

'Thursday, July 23rd at 09:30'

In [125]:
tracks = []
for session_name in session_names:
    talks = talk_sessions[session_name]
    talks = [talks[talk_id] for talk_id in talks]
    talks = sorted(talks, key=itemgetter('title'))
    for talk in talks:
        tracks.append(talk["track_title"])
tracks = set(tracks)
print(tracks)

set([u'', u'Barria2 Room (Education Summit)', u'Barria2 Room', u'Room A4', u'A2 Room', u'Room B Terrace', u'Room C1', u'Barria1 Room', u'PythonAnywhere Room', u'Google Room', u'Exhibition Hall / Helpdesk ', u'Barria2 Room (Local Track)'])
